In [7]:
import refinitiv.data as rd
import pandas as pd
import numpy as np
import cufflinks as cf
import re
from datetime import datetime, timedelta
import plotly.graph_objects as go

rd.open_session()


<refinitiv.data.session.Definition object at 0x14fcfc340 {name='workspace'}>

In [15]:
def refinitiv_layout(figure):
    figure.update_layout(margin=dict(t=5, b=5, l=10, r=10),
                         plot_bgcolor='#1A1A1D',
                         paper_bgcolor='#1A1A1D',
                         yaxis=dict(gridcolor='black', gridwidth=1, zerolinecolor='black', zerolinewidth=1,
                                      side='right', color='#D6D6D5', linecolor='#D6D6D5', ticks='outside'),
                         xaxis=dict(gridcolor='black', gridwidth=1, zerolinecolor='black', zerolinewidth=1,
                                    color='#D6D6D5', linecolor='#D6D6D5', ticks='outside'),
                         colorway=['#6978F7', '#A325E9', '#96E05D', '#4A7FB9',
                                   '#E75A2D', '#FBE55A', '#8C8C8D', '#5A54F6'],
                         legend=dict(font=dict(color='#D6D6D5'), orientation='v',
                                     yanchor='top', xanchor='center', y=1.1, x=0.1),
                         title=dict(xanchor='center', yanchor='top',
                                    x=0.5, font=dict(color='#D6D6D5'))
                         )
    return


def get_data(fx_list, _base_currency):
    usd_fx_list = list(map(lambda x: x + '=', fx_list))
    df_snap = rd.get_data(usd_fx_list, 'BID')
    df_snap = df_snap.sort_values(ascending=False, by='BID')
    fx_list = list(map(lambda x: re.sub(r"=", "", x),
                   df_snap.Instrument.tolist()))

    xfx_list = []
    base_currency = _base_currency + '='
    for f in fx_list:
        for _f in fx_list:
            if f == _f:
                break
            else:
                xfx_list.append(f + _f + '=R')

    xfx_list.append(base_currency)
    start = datetime.today() - timedelta(days=91)
    df = rd.get_history(xfx_list, 'BID', start=start,
                        interval='daily', count=10000)
    df_base = df[base_currency]

    df_log = pd.DataFrame()
    for c in list(df.columns):
        df_log[c] = np.log(df[c]) - np.log(df[c].shift(1))
    df_log = df_log.dropna(axis=0)
    df_mean = df.mean()
    df_vol_absolute = df.std()
    df_z_scores = (df.tail(1) - df_mean) / df_vol_absolute
    df_corr = df_log.corr()
    corr_values = df_corr[base_currency][:-1].values.tolist()
    z_score_values = df_z_scores.drop(
        columns=[base_currency]).values[0].tolist()
    df_out = pd.DataFrame(
        columns=['91D Z-Scores', 'Correlation vs ' + _base_currency])
    df_out['91D Z-Scores'] = z_score_values
    df_out['Correlation vs ' + _base_currency] = corr_values
    df_out.index = list(map(lambda x: re.sub(r"=R", "", x), xfx_list[:-1]))
    df_out.index = list(map(lambda x: x[:3] + '/' + x[3:], df_out.index))
    trace = go.Scatter(
        mode='markers',
        x=df_out['91D Z-Scores'],
        y=df_out['Correlation vs ' + _base_currency],
        text=df_out.index.tolist(),
        marker=dict(
            size=7,
            color=np.abs((df_out['91D Z-Scores']).astype(float)),
            colorscale='Sunsetdark'
        )
    )

    fig = go.Figure()
    fig.layout.title = 'Currency relative strength'
    fig.add_trace(trace)
    fig.layout.xaxis.title = '91D Z-Scores'
    fig.layout.yaxis.title = 'Correlation vs ' + _base_currency
    refinitiv_layout(fig)
    fig.show()
    return df_out

In [6]:
rd.close_session()
